In [57]:
#Script for simulating data from model

In [3]:
import os
from pathlib import Path

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [66]:
import numpy as np
import pandas as pd

import json

import statsmodels.api as sm
from statsmodels.tsa.filters.hp_filter import hpfilter

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import kaiming_uniform_

import matplotlib.pyplot as plt
from matplotlib import cm

In [5]:
DEVICE = torch.device(os.getenv('OP_DEVICE', 'cuda' if torch.cuda.is_available() else 'cpu'))
DTYPE  = torch.float64


In [10]:
weight = 0.7
neg_weight = 10.0
batch_size = 128

layers = [50, 50, 50]
seed = 33

In [11]:
torch.manual_seed(seed)            
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)    
    torch.cuda.manual_seed_all(seed)   


In [23]:
# -----------------------------------------------------------------------------
#  Hard‑wired model parameters
# -----------------------------------------------------------------------------
crra = 5.0
beta = 0.9
alpha = 0.5

psi = 7.08
pi_star = 1.0484

rho_y = 0.8118
eta = 0.0347
sigma_y = eta/np.sqrt(1.0 - (rho_y ** 2.0))

mu = 0.0
y_ubnd = mu + 3.0 * sigma_y
y_lbnd = mu - 3.0 * sigma_y

b_star_ubnd = 0.3
b_star_lbnd = -0.1

b_tilde_ubnd = 0.3
b_tilde_lbnd = -0.1

kappa = 40.0

r = 0.039
R = np.exp(r)
delta = 0.757

qstar = 1.0 / (R - delta)

In [36]:
path = ''
base_path = f'{path}/pickle/kappa{kappa}'

In [37]:
n_int = 19

x_int_norm, w_int = np.polynomial.hermite.hermgauss(n_int)
w_int = w_int/np.sqrt(np.pi)
x_int_norm = x_int_norm * np.sqrt(2)

w_int = torch.from_numpy(w_int).to(DEVICE)
x_int_norm = torch.from_numpy(x_int_norm).to(DEVICE)

In [38]:
class NN_pi(nn.Module):
    def __init__(self, in_szs, out_szs, layers):
        super().__init__()

        layerlist = []
        n_in = in_szs
        
        for i in layers:
            kaiming_uniform_(nn.Linear(n_in, i).weight, nonlinearity='leaky_relu')
            layerlist.append(nn.Linear(n_in, i))
            layerlist.append(nn.LeakyReLU())
            n_in = i
        
        layerlist.append(nn.Linear(layers[-1], out_szs))
        layerlist.append(nn.Sigmoid())
        
        self.layers = nn.Sequential(*layerlist)
                
    def forward(self, x):
        
        pred = self.layers(x)
        
        pred = pred * (1.1 - 1.0) + 1.0
                    
        return pred

In [39]:
class NN_foreign(nn.Module):
    def __init__(self, in_szs, out_szs, layers):
        super().__init__()

        layerlist = []
        n_in = in_szs
        
        for i in layers:
            kaiming_uniform_(nn.Linear(n_in, i).weight, nonlinearity='leaky_relu')
            layerlist.append(nn.Linear(n_in, i))
            layerlist.append(nn.LeakyReLU())
            n_in = i
        
        layerlist.append(nn.Linear(layers[-1], out_szs))
        layerlist.append(nn.Identity())
        
        self.layers = nn.Sequential(*layerlist)
                
    def forward(self, x):
        
        pred = self.layers(x)
                    
        return pred

In [40]:
class NN_local(nn.Module):
    def __init__(self, in_szs, out_szs, layers):
        super().__init__()

        layerlist = []
        n_in = in_szs
        
        for i in layers:
            kaiming_uniform_(nn.Linear(n_in, i).weight, nonlinearity='leaky_relu')
            layerlist.append(nn.Linear(n_in, i))
            layerlist.append(nn.LeakyReLU())
            n_in = i
        
        layerlist.append(nn.Linear(layers[-1], out_szs))
        layerlist.append(nn.Identity())
        
        self.layers = nn.Sequential(*layerlist)
                
    def forward(self, x):
        
        pred = self.layers(x)
                    
        return pred

In [41]:
class NN_val(nn.Module):
    def __init__(self, in_szs, out_szs, layers):
        super().__init__()

        layerlist = []
        n_in = in_szs
        
        for i in layers:
            kaiming_uniform_(nn.Linear(n_in, i).weight, nonlinearity='leaky_relu')
            layerlist.append(nn.Linear(n_in, i))
            layerlist.append(nn.LeakyReLU())
            n_in = i
        
        layerlist.append(nn.Linear(layers[-1], out_szs))
        layerlist.append(nn.Identity())
        
        self.layers = nn.Sequential(*layerlist)
                
    def forward(self, x):
        
        pred = self.layers(x)
                    
        return pred

In [42]:
class NN_qtilde(nn.Module):
    def __init__(self, in_szs, out_szs, layers):
        super().__init__()

        layerlist = []
        n_in = in_szs
        
        for i in layers:
            kaiming_uniform_(nn.Linear(n_in, i).weight, nonlinearity='leaky_relu')
            layerlist.append(nn.Linear(n_in, i))
            layerlist.append(nn.LeakyReLU())
            n_in = i
        
        layerlist.append(nn.Linear(layers[-1], out_szs))
        layerlist.append(nn.Sigmoid())
        
        self.layers = nn.Sequential(*layerlist)
                
    def forward(self, x):
        
        pred = self.layers(x)
        
        pred = pred * 10.0
                    
        return pred

In [43]:
class NN_q(nn.Module):
    def __init__(self, in_szs, out_szs, layers):
        super().__init__()

        layerlist = []
        n_in = in_szs
        
        for i in layers:
            kaiming_uniform_(nn.Linear(n_in, i).weight, nonlinearity='leaky_relu')
            layerlist.append(nn.Linear(n_in, i))
            layerlist.append(nn.LeakyReLU())
            n_in = i
        
        layerlist.append(nn.Linear(layers[-1], out_szs))
        layerlist.append(nn.Softplus())
        
        self.layers = nn.Sequential(*layerlist)
                
    def forward(self, x):
        
        pred = self.layers(x)
                                    
        return pred

In [44]:
new_model_pi = NN_pi(3, 1, layers)
new_model_pi.load_state_dict(torch.load(f'{base_path}_Model_pi.pt', map_location=DEVICE))
new_model_pi = new_model_pi.to(dtype = torch.float64).to(DEVICE)
new_model_pi.eval()

NN_pi(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=50, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [45]:
new_model_foreign = NN_foreign(3, 1, layers)
new_model_foreign.load_state_dict(torch.load(f'{base_path}_Model_foreign.pt', map_location=DEVICE))
new_model_foreign = new_model_foreign.to(dtype = torch.float64).to(DEVICE)
new_model_foreign.eval()

NN_foreign(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=50, out_features=1, bias=True)
    (7): Identity()
  )
)

In [46]:
new_model_local = NN_local(3, 1, layers)
new_model_local.load_state_dict(torch.load(f'{base_path}_Model_local.pt', map_location=DEVICE))
new_model_local = new_model_local.to(dtype = torch.float64).to(DEVICE)
new_model_local.eval()

NN_local(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=50, out_features=1, bias=True)
    (7): Identity()
  )
)

In [47]:
new_model_val = NN_val(3, 1, layers)
new_model_val.load_state_dict(torch.load(f'{base_path}_Model_val.pt', map_location=DEVICE))
new_model_val = new_model_val.to(dtype = torch.float64).to(DEVICE)
new_model_val.eval()

NN_val(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=50, out_features=1, bias=True)
    (7): Identity()
  )
)

In [48]:
new_model_qtilde = NN_qtilde(3, 1, layers)
new_model_qtilde.load_state_dict(torch.load(f'{base_path}_Model_qtilde.pt', map_location=DEVICE))
new_model_qtilde = new_model_qtilde.to(dtype = torch.float64).to(DEVICE)
new_model_qtilde.eval()

NN_qtilde(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=50, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [49]:
new_model_q = NN_q(3, 1, layers)
new_model_q.load_state_dict(torch.load(f'{base_path}_Model_q.pt', map_location=DEVICE))
new_model_q = new_model_q.to(dtype = torch.float64).to(DEVICE)
new_model_q.eval()

NN_q(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=50, out_features=1, bias=True)
    (7): Softplus(beta=1, threshold=20)
  )
)

In [50]:
n_data = 16

y_t_grid_log = torch.linspace(y_lbnd, y_ubnd, n_data)
y_t_grid = torch.exp(y_t_grid_log).to(dtype = torch.float64).to(DEVICE)

bstar_grid = torch.linspace(b_star_lbnd, b_star_ubnd, n_data).to(dtype = torch.float64).to(DEVICE)

btilde_grid = torch.linspace(b_tilde_lbnd, b_tilde_ubnd, n_data).to(dtype = torch.float64).to(DEVICE)


In [51]:
def single_step(X, eps, nn_pi, nn_foreign, nn_local, nn_val, nn_qtilde, nn_q):
    
    n_data = X.shape[0]
    
    yt = X[0:1]
    bstar = X[1:2]
    btilde = X[2:3]
    
    with torch.no_grad():
        pi = nn_pi(X)
        bstar_prime = nn_foreign(X)
        btilde_prime = nn_local(X)
        val = nn_val(X)
    
    yt_prime = torch.exp(rho_y * torch.log(yt) + eps * eta)
    
    q_input = torch.cat((yt, bstar_prime, btilde_prime))
    
    with torch.no_grad():
        qtilde = nn_qtilde(q_input)
        q = nn_q(q_input)
    
    ct_discriminant = ((btilde / pi) ** 2.0) - 4.0 * (bstar - qstar * (bstar_prime - delta * bstar) - qtilde * (btilde_prime - delta * (btilde / pi)) - yt)
    ct_discriminant_clamped = torch.clamp(ct_discriminant, min = 10e-12)
    numerator = (-btilde / pi) + (ct_discriminant_clamped ** 0.5)
    ct = 0.25 * (numerator ** 2.0)
    
    zeta = (1.0 / pi) * (ct ** (1.0 - alpha))
    
    X_prime = torch.cat((yt_prime, bstar_prime, btilde_prime))
                         
    return X_prime, pi, ct, qtilde, q, bstar_prime, btilde_prime
    

In [52]:
def sim_data(num_periods):

    X_sim = torch.empty((num_periods, 3)).to(dtype = torch.float64).to(DEVICE)
    pi_sim = torch.empty((num_periods, 1)).to(dtype = torch.float64).to(DEVICE)
    ct_sim = torch.empty((num_periods, 1)).to(dtype = torch.float64).to(DEVICE)
    qtilde_sim = torch.empty((num_periods, 1)).to(dtype = torch.float64).to(DEVICE)
    q_sim = torch.empty((num_periods, 1)).to(dtype = torch.float64).to(DEVICE)  
    bstar_sim = torch.empty((num_periods, 1)).to(dtype = torch.float64).to(DEVICE) 
    btilde_sim = torch.empty((num_periods, 1)).to(dtype = torch.float64).to(DEVICE) 
    eps = torch.randn(num_periods, dtype = torch.float64).to(DEVICE)

    X_sim[0,0] = torch.mean(y_t_grid)
    X_sim[0,1] = torch.mean(bstar_grid)
    X_sim[0,2] = torch.mean(btilde_grid)

    X_old = X_sim[0, :]

    for i in range(0, num_periods-1):

        eps_use = eps[i]

        X_new, pi, ct, qtilde, q, bstar_prime, btilde_prime = single_step(X_old, eps_use, new_model_pi, new_model_foreign, new_model_local, new_model_val, new_model_qtilde, new_model_q)

        X_sim[i, :] = X_new

        pi_sim[i] = pi
        ct_sim[i] = ct
        qtilde_sim[i] = qtilde
        q_sim[i] = q
        bstar_sim[i] = bstar_prime
        btilde_sim[i] = btilde_prime
        
        X_old = X_new

        if i%5000 == 0:
                print(f'Simulated {i} periods of data')
    
    sim_data = torch.cat((X_sim, pi_sim, ct_sim, qtilde_sim, q_sim, bstar_sim, btilde_sim), dim = 1)
    
    return sim_data
      

In [53]:
sim_full = sim_data(50000)

Simulated 0 periods of data
Simulated 5000 periods of data
Simulated 10000 periods of data
Simulated 15000 periods of data
Simulated 20000 periods of data
Simulated 25000 periods of data
Simulated 30000 periods of data
Simulated 35000 periods of data
Simulated 40000 periods of data
Simulated 45000 periods of data


In [54]:
sim_full.shape

torch.Size([50000, 9])

In [55]:
drop = 5000

In [56]:
yt_sim = sim_full[drop:-1, 0].cpu().detach().numpy()
dstar_sim = sim_full[drop:-1, 1].cpu().detach().numpy()
dtilde_sim = sim_full[drop:-1, 2].cpu().detach().numpy()
pi_sim = sim_full[drop:-1, 3].cpu().detach().numpy()
ct_sim = sim_full[drop:-1, 4].cpu().detach().numpy()
qtilde_sim = sim_full[drop:-1, 5].cpu().detach().numpy()
q_sim = sim_full[drop:-1, 6].cpu().detach().numpy()
dstar_prime_sim = sim_full[drop:-1, 7].cpu().detach().numpy()
dtilde_prime_sim = sim_full[drop:-1, 8].cpu().detach().numpy()

In [57]:
#Construct btilde_sim and bstar_sim
zeta_sim = (1.0 / pi_sim) * (ct_sim ** (1.0 - alpha))
bstar_sim = dstar_sim
btilde_sim = dtilde_sim * zeta_sim

bt_sim = bstar_sim + btilde_sim

pn_sim = ((1 - alpha) / alpha) * ct_sim

#Nominal GDP in dollars
gdpt_sim = yt_sim + pn_sim

gdpr_sim = (alpha ** alpha) * ((1.0 - alpha) ** (1.0 - alpha)) * (yt_sim * (pn_sim ** (alpha - 1.0)) + (pn_sim ** alpha))

LC_dif_sim = (btilde_sim - bstar_sim) / gdpt_sim

LC_rat_sim = btilde_sim / bt_sim

debt_gdpt_sim = bt_sim / gdpt_sim

debtLC_gdpt_sim = btilde_sim / gdpt_sim

exchange_rate_sim = alpha * pi_sim * (ct_sim ** (alpha - 1.0))

trade_balance_sim = (yt_sim - ct_sim) / gdpt_sim

LC_yield_sim = np.log(delta + (1.0 / q_sim))
LC_spread_sim = LC_yield_sim - r

In [58]:
#Set smoothing parameter to annual frequency. Model period corresponds to one year
lambda_hp = 6.25

In [59]:
# -----------------------------------------------------------------------------
#  HP-filtered series
# -----------------------------------------------------------------------------

def hp_filtered(series, lam):
    cycle, trend = hpfilter(series, lamb = lam)
    
    return cycle

hp_externaldebt_gdp_sim = hp_filtered(debt_gdpt_sim, lambda_hp)

hp_share_LC_rat_sim = hp_filtered(LC_rat_sim, lambda_hp)

hp_share_LC_dif_sim = hp_filtered(LC_dif_sim, lambda_hp)

hp_pi_sim = hp_filtered(pi_sim, lambda_hp)

hp_pn_sim = hp_filtered(pn_sim, lambda_hp)

hp_exchange_sim = np.log(exchange_rate_sim)
hp_exchange_sim = hp_filtered(hp_exchange_sim, lambda_hp)

hp_yt_sim = np.log(yt_sim)
hp_yt_sim = hp_filtered(hp_yt_sim, lambda_hp)

hp_gdpr_sim = np.log(gdpr_sim)
hp_gdpr_sim = hp_filtered(hp_gdpr_sim, lambda_hp)

hp_gdpt_sim = np.log(gdpt_sim)
hp_gdpt_sim = hp_filtered(hp_gdpt_sim, lambda_hp)

hp_trade_bal_sim = hp_filtered(trade_balance_sim, lambda_hp)

hp_LC_yield_sim = hp_filtered(LC_yield_sim, lambda_hp)
hp_LC_spread_sim = hp_filtered(LC_spread_sim, lambda_hp)


In [60]:
# -----------------------------------------------------------------------------
#  Compute first moments
# -----------------------------------------------------------------------------

av_externaldebt_gdp = np.mean(debt_gdpt_sim)

av_share_LC = np.mean(debtLC_gdpt_sim) / np.mean(debt_gdpt_sim)

av_pi = np.mean(pi_sim)

av_xr_sim = np.mean(exchange_rate_sim)

av_dstar_sim = np.mean(dstar_sim)

av_dtilde_sim = np.mean(dtilde_sim)

av_zeta_sim = np.mean(zeta_sim)

av_LC_yield_sim = np.mean(LC_yield_sim)

av_LC_spread_sim = np.mean(LC_spread_sim)

In [61]:
# -----------------------------------------------------------------------------
#  Compute standard deviations
# -----------------------------------------------------------------------------
sd_externaldebt_gdp = np.std(debt_gdpt_sim)

sd_share_LC_rat = np.std(LC_rat_sim)

sd_share_LC_dif = np.std(LC_dif_sim)

sd_pi = np.std(pi_sim)

sd_pn = np.std(pn_sim)

sd_exchange_rate = np.std(np.log(exchange_rate_sim))

sd_yt = np.std(yt_sim)

sd_gdpr = np.std(gdpr_sim)

sd_gdpt = np.std(gdpt_sim)

sd_LC_yield = np.std(LC_yield_sim)

sd_LC_spread = np.std(LC_spread_sim)

sd_externaldebt_gdp_hp = np.std(hp_externaldebt_gdp_sim)

sd_share_LC_rat_hp = np.std(hp_share_LC_rat_sim)

sd_share_LC_dif_hp = np.std(hp_share_LC_dif_sim)

sd_pi_hp = np.std(hp_pi_sim)

sd_pn_hp = np.std(hp_pn_sim)

sd_exchange_hp = np.std(hp_exchange_sim)

sd_yt_hp = np.std(hp_yt_sim)

sd_gdpr_hp = np.std(hp_gdpr_sim)

sd_gdpt_hp = np.std(hp_gdpt_sim)

sd_tby_hp = np.std(hp_trade_bal_sim)

sd_LC_yield_hp = np.std(hp_LC_yield_sim)

sd_LC_spread_hp = np.std(hp_LC_spread_sim)


In [62]:
# -----------------------------------------------------------------------------
#  Compute correlations
# -----------------------------------------------------------------------------

def corr(x, y):
    return np.corrcoef(x, y)[0,1]

corr_hp_gdp_externaldebt_gdp = corr(hp_gdpr_sim, hp_externaldebt_gdp_sim)

corr_hp_gdp_share_LC_rat = corr(hp_gdpr_sim, hp_share_LC_rat_sim)

corr_hp_gdp_share_LC_dif = corr(hp_gdpr_sim, hp_share_LC_dif_sim)

corr_hp_gdp_pi = corr(hp_gdpr_sim, hp_pi_sim)

corr_hp_gdp_exchange = corr(hp_gdpr_sim, hp_exchange_sim)

corr_hp_gdp_tby = corr(hp_gdpr_sim, hp_trade_bal_sim)

corr_hp_gdp_LC_yield = corr(hp_gdpr_sim, hp_LC_yield_sim)

corr_hp_gdp_LC_spread = corr(hp_gdpr_sim, hp_LC_spread_sim)

# Correlations with yt
corr_hp_yt_externaldebt_gdp = corr(hp_yt_sim, hp_externaldebt_gdp_sim)

corr_hp_yt_share_LC_rat = corr(hp_yt_sim, hp_share_LC_rat_sim)

corr_hp_yt_share_LC_dif = corr(hp_yt_sim, hp_share_LC_dif_sim)

corr_hp_yt_pi = corr(hp_yt_sim, hp_pi_sim)

corr_hp_yt_pn = corr(hp_yt_sim, hp_pn_sim)

corr_hp_yt_exchange = corr(hp_yt_sim, hp_exchange_sim)

corr_hp_yt_tby = corr(hp_yt_sim, hp_trade_bal_sim)

corr_hp_yt_LC_yield = corr(hp_yt_sim, hp_LC_yield_sim)

corr_hp_yt_LC_spread = corr(hp_yt_sim, hp_LC_spread_sim)

# Correlations with gdpt
corr_hp_gdpt_externaldebt_gdp = corr(hp_gdpt_sim, hp_externaldebt_gdp_sim)

corr_hp_gdpt_share_LC_rat = corr(hp_gdpt_sim, hp_share_LC_rat_sim)

corr_hp_gdpt_share_LC_dif = corr(hp_gdpt_sim, hp_share_LC_dif_sim)

corr_hp_gdpt_pi = corr(hp_gdpt_sim, hp_pi_sim)

corr_hp_gdpt_exchange = corr(hp_gdpt_sim, hp_exchange_sim)

corr_hp_gdpt_tby = corr(hp_gdpt_sim, hp_trade_bal_sim)

corr_hp_gdpt_LC_yield = corr(hp_gdpt_sim, hp_LC_yield_sim)

corr_hp_gdpt_LC_spread = corr(hp_gdpt_sim, hp_LC_spread_sim)


In [63]:
# -----------------------------------------------------------------------------
#  Store moments in dictionary
# -----------------------------------------------------------------------------

moments = {
    # Averages
    'av_externaldebt_gdp': av_externaldebt_gdp,
    'av_share_LC': av_share_LC,
    'av_pi': av_pi,
    'av_LC_yield_sim': av_LC_yield_sim,
    'av_LC_spread_sim':av_LC_spread_sim,
    
    # Standard deviations (levels)
    'sd_externaldebt_gdp': sd_externaldebt_gdp,
    'sd_share_LC_rat': sd_share_LC_rat,
    'sd_share_LC_dif': sd_share_LC_dif,
    'sd_pi': sd_pi,
    'sd_exchange_rate': sd_exchange_rate,
    'sd_yt': sd_yt,
    'sd_gdpr': sd_gdpr,
    'sd_gdpt': sd_gdpt,
    'sd_LC_yield': sd_LC_yield,
    'sd_LC_spread' : sd_LC_spread,
    
    # Standard deviations (HP-filtered series)
    'sd_externaldebt_gdp_hp': sd_externaldebt_gdp_hp,
    'sd_share_LC_rat_hp': sd_share_LC_rat_hp,
    'sd_share_LC_dif_hp': sd_share_LC_dif_hp,
    'sd_pi_hp': sd_pi_hp,
    'sd_exchange_hp': sd_exchange_hp,
    'sd_yt_hp': sd_yt_hp,
    'sd_gdpr_hp': sd_gdpr_hp,
    'sd_gdpt_hp' : sd_gdpt_hp,
    'sd_tby_hp': sd_tby_hp,
    'sd_LC_yield_hp':sd_LC_yield_hp,
    'sd_LC_spread_hp': sd_LC_spread_hp,
    
    # Correlations between HP-filtered GDP and other variables
    'corr_hp_gdp_externaldebt_gdp': corr_hp_gdp_externaldebt_gdp,
    'corr_hp_gdp_share_LC_rat': corr_hp_gdp_share_LC_rat,
    'corr_hp_gdp_share_LC_dif': corr_hp_gdp_share_LC_dif,
    'corr_hp_gdp_pi': corr_hp_gdp_pi,
    'corr_hp_gdp_exchange': corr_hp_gdp_exchange,
    'corr_hp_gdp_tby': corr_hp_gdp_tby,
    'corr_hp_gdp_LC_yield': corr_hp_gdp_LC_yield,
    'corr_hp_gdp_LC_spread': corr_hp_gdp_LC_spread,
    
    # Correlations between HP-filtered yt and other variables
    'corr_hp_yt_externaldebt_gdp': corr_hp_yt_externaldebt_gdp,
    'corr_hp_yt_share_LC_rat': corr_hp_yt_share_LC_rat,
    'corr_hp_yt_share_LC_dif': corr_hp_yt_share_LC_dif,
    'corr_hp_yt_pi': corr_hp_yt_pi,
    'corr_hp_yt_exchange': corr_hp_yt_exchange,
    'corr_hp_yt_tby': corr_hp_yt_tby,
    'corr_hp_yt_LC_yield' : corr_hp_yt_LC_yield,
    'corr_hp_yt_LC_spread': corr_hp_yt_LC_spread
}


In [64]:
moments

{'av_externaldebt_gdp': 0.03371833776683496,
 'av_share_LC': -0.6121440400197636,
 'av_pi': 1.0520455316624227,
 'av_xr_sim': 0.5363314677674444,
 'av_dstar_sim': 0.10662147467615092,
 'av_dtilde_sim': -0.043425937056155856,
 'av_zeta_sim': 0.9331864994386172,
 'av_LC_yield_sim': 0.7510227243771228,
 'av_LC_spread_sim': 0.7120227243771228,
 'sd_externaldebt_gdp': 0.0018453282366034255,
 'sd_share_LC_rat': 0.026162065380063146,
 'sd_share_LC_dif': 0.002502802961343695,
 'sd_pi': 5.492797744359008e-05,
 'sd_pn': 0.061077967157454616,
 'sd_exchange_rate': 0.03152902077385794,
 'sd_yt': 0.059251506423559905,
 'sd_gdpr': 0.029675673947454836,
 'sd_gdpt': 0.11442825496500131,
 'sd_LC_yield': 0.00011231206078204429,
 'sd_LC_spread': 0.00011231206078204429,
 'sd_externaldebt_gdp_hp': 0.0006051084721633818,
 'sd_share_LC_rat_hp': 0.01151251347737062,
 'sd_share_LC_dif_hp': 0.000932022249170027,
 'sd_pi_hp': 2.0409924346244485e-05,
 'sd_pn_hp': 0.02574962335996252,
 'sd_exchange_hp': 0.013238881

In [68]:
base = Path(path or ".").expanduser().resolve()
outfile = base / "moments.json"
outfile.write_text(json.dumps(moments, indent=2, sort_keys=True))

2074